In [1]:
import sympy as sp
from gEcon import *
from scipy import optimize
from model_utilities import *
from TimeAwareSympy import *

In [2]:
model_file = 'GCN Files/Test_Model.gcn'

In [37]:
model, param_dict, shocks, var_list, try_reduce_vars = parse_gcn_file(model_file)
subbed_model = substitute_all_params(model, param_dict)
subbed_model = rewrite_model_equations_eq_0(subbed_model)
model = rewrite_model_equations_eq_0(model)

print(f'n equations: {len(subbed_model)}')
print(f'''n variables: {len(set([x.base_name if isinstance(x, TimeAwareSymbol)
                                else x.name for x in var_list]))}''')

n equations: 10
n variables: 10


In [38]:
model_ss, var_list_ss = convert_model_to_steady_state(subbed_model, var_list)
f_model, f_jac = convert_model_to_numpy_function(model_ss, var_list_ss)

In [39]:
results, ss_value_dict = compute_ss(f_model, var_list_ss, f_jac=f_jac)

6       / 10000
Done!


In [41]:
for eq in model:
    display(eq)

-beta*U_t+1 + U_t - (C_t**theta*(1 - L_t)**(1 - theta))**(1 - tau)/(1 - tau)

-A_t*K_t**alpha*L_t**(1 - alpha) + Y_t

C_t + I_t - Y_t

-I_t + K_t - K_t-1*(1 - delta)

-rho*log(A_t-1) - epsilon_t + log(A_t)

theta*(C_t**theta*(1 - L_t)**(1 - theta))**(1 - tau)/C_t - lambda__H1_t

A_t*K_t**alpha*L_t**(-alpha)*lambda_t*(1 - alpha) + (C_t**theta*(1 - L_t)**(1 - theta))**(1 - tau)*(theta - 1)/(1 - L_t)

-lambda__H1_t + q_t

alpha*A_t*K_t**(alpha - 1)*L_t**(1 - alpha)*lambda_t + beta*q_t+1*(1 - delta) - q_t

lambda__H1_t - lambda_t

In [29]:
def make_loglin_sub_dict(var_list):
    sub_dict = {}
    for var in var_list:
        ss_var = var.to_ss()
        var_tile = var  
        sub_dict[var] = ss_var * sp.exp(var / ss_var)
        
    return sub_dict

In [92]:
import uuid

In [136]:
def build_hash_table(model_things):
    var_to_hash = {}
    hash_to_var = {}
    name_list = [x.name for x in model_things]
    for thing in sorted(name_list, key=len, reverse=True):
        hashkey = str(uuid.uuid4().int)
        var_to_hash[thing] = hashkey
        hash_to_var[hashkey] = thing
        
    return var_to_hash, hash_to_var

In [137]:
def sub_things_to_hash(eq_str, hash_dict):
    for key, value in hash_dict.items():
        eq_str = eq_str.replace(key, value)
    return eq_str

In [138]:
operators = list('+-/*^()=')

In [171]:
from sympy.abc import greeks, _clash, _clash1, _clash2

In [191]:
def get_matlab_safe_unique_vars_from_list(var_list):
    unique_vars = [var for var in set([x.base_name if isinstance(x, TimeAwareSymbol) else x.name\
                                        for x in var_list])]
    unique_vars = [var if var.lower() not in greeks else 'a' + var for var in unique_vars]
    
    return unique_vars

def convert_var_timings_to_matlab(var_list):
    matlab_var_list = [var.replace('_t+1', '(1)').replace('_t-1', '(-1)').replace('_t', '')\
                      for var in var_list]
    
    return matlab_var_list

In [223]:
def make_dynare_file(model, var_list, param_dict, shocks, ss_value_dict):
    dynare_vars = get_matlab_safe_unique_vars_from_list(var_list)
#     dynare_vars = convert_var_timings_to_matlab(unique_vars)
    
    file = ''
    line = 'var'
    for var in dynare_vars:
        line += f' {var},'
        if len(line) > 50:
            line = line[:-1]
            line += line + ';\n'
            file += line
            line = 'var'
    line = line[:-1]
    file += line + ';\n'
    
    unique_shocks = get_matlab_safe_unique_vars_from_list(shocks)
#     dynare_shocks = convert_var_timings_to_matlab(shocks)
    line = 'varexo'
    for shock in unique_shocks:
        line += f' {shock}'
        if len(line) > 50:
            line += line + ';\n'
            file += line
            line = 'varexo'
    file += line + ';\n'
    file += '\n'
    
    dynare_param_names = get_matlab_safe_unique_vars_from_list(param_dict.keys())
    file += 'parameters\n'
    for param, value in zip(dynare_param_names, param_dict.values()):
        file += f'{param} = {value};\n'
        
        
    file +='\n'
    file +='model;\n'
    
    for eq in model:
        eq_str = str(eq)
        prepare_eq = eq_str.replace('**', '^')
        var_to_hash, hash_to_var = build_hash_table(model_things)

        hash_eq = sub_things_to_hash(prepare_eq, var_to_hash)
        for operator in operators:
            hash_eq = hash_eq.replace(operator, ' ' + operator  + ' ')
        hash_eq = re.sub(' +', ' ', hash_eq)
        hash_eq = hash_eq.strip()
        final_eq = sub_things_to_hash(hash_eq, hash_to_var)
        matlab_eq = final_eq.replace('_t+1', '(1)').replace('_t-1', '(-1)').replace('_t', '')
        split_eq = matlab_eq.split(' ')
        new_eq = []
        for atom in split_eq:
            if atom.lower() in greeks:
                atom = 'a' + atom
            new_eq.append(atom)
        matlab_eq = ''.join(atom for atom in new_eq) + ';'
        file += matlab_eq + "\n"
    
    file += 'end;\n\n'
    
    file += 'initval;\n'
    
    for var, val in ss_value_dict.items():
        matlab_var = get_matlab_safe_unique_vars_from_list([var])[0]
        file += f'{matlab_var} = {val:0.4f};\n'
    
    file += 'end;\n\n'
    print(file)

In [224]:
make_dynare_file(model, var_list, param_dict, shocks, ss_value_dict)

var A, U, lambda__H1, q, L, alambda, K, C, I, Y;
varexo aepsilon;

parameters
arho = 0;
aalpha = 0.4;
adelta = 0.357;
atheta = 0.99;
abeta = 0.02;
atau = 2.0;
aepsilon = 0.95;

model;
-abeta*U(1)+U-(C^atheta*(1-L)^(1-atheta))^(1-atau)/(1-atau);
-A*K^aalpha*L^(1-aalpha)+Y;
C+I-Y;
-I+K-K(-1)*(1-adelta);
-arho*log(A(-1))-aepsilon+log(A);
atheta*(C^atheta*(1-L)^(1-atheta))^(1-atau)/C-lambda__H1;
A*K^aalpha*L^(-aalpha)*alambda*(1-aalpha)+(C^atheta*(1-L)^(1-atheta))^(1-atau)*(atheta-1)/(1-L);
-lambda__H1+q;
aalpha*A*K^(aalpha-1)*L^(1-aalpha)*alambda+abeta*q(1)*(1-adelta)-q;
lambda__H1-alambda;
end;

initval;
U = -116.1386;
L = 0.3129;
I = 0.4744;
K = 23.7210;
Y = 1.7672;
C = 1.2928;
alambda = 0.3207;
lambda__H1 = 0.3207;
q = 0.3207;
A = 1.0000;
end;




In [150]:
for eq in model:
    test_str = str(eq)
    prepare_eq = test_str.replace('**', '^')
    var_to_hash, hash_to_var = build_hash_table(model_things)

    hash_eq = sub_things_to_hash(prepare_eq, var_to_hash)
    for operator in operators:
        hash_eq = hash_eq.replace(operator, ' ' + operator  + ' ')
    hash_eq = re.sub(' +', ' ', hash_eq)
    hash_eq = hash_eq.strip()
    final_eq = sub_things_to_hash(hash_eq, hash_to_var)
    matlab_eq = final_eq.replace('_t+1', '(1)').replace('_t-1', '(-1)').replace('_t', '')
    split_eq = matlab_eq.split(' ')
    new_eq = []
    for atom in split_eq:
        if atom.lower() in greeks:
            atom = 'a' + atom
        new_eq.append(atom)
    matlab_eq = ''.join(atom for atom in new_eq) + ';'
    print(matlab_eq)

-abeta*U(1)+U-(C^atheta*(1-L)^(1-atheta))^(1-atau)/(1-atau);
-A*K^aalpha*L^(1-aalpha)+Y;
C+I-Y;
-I+K-K(-1)*(1-adelta);
-arho*log(A(-1))-aepsilon+log(A);
atheta*(C^atheta*(1-L)^(1-atheta))^(1-atau)/C-lambda__H1;
A*K^aalpha*L^(-aalpha)*alambda*(1-aalpha)+(C^atheta*(1-L)^(1-atheta))^(1-atau)*(atheta-1)/(1-L);
-lambda__H1+q;
aalpha*A*K^(aalpha-1)*L^(1-aalpha)*alambda+abeta*q(1)*(1-adelta)-q;
lambda__H1-alambda;


In [140]:
final_eq

'- beta * U_t+1  +  U_t  -   ( C_t ^ theta *  ( 1  -  L_t )  ^  ( 1  -  theta )  )  ^  ( 1  -  tau )  /  ( 1  -  tau )'

In [85]:
spaced_eq = test_str.replace('**', ' ^ ').replace('*', ' * ').replace('/', ' / ')
spaced_eq = spaced_eq.replace('(', ' ( ').replace(')', ' ) ')

In [88]:
model_things = var_list + list(param_dict.keys()) + shocks

In [86]:
spaced_eq

'-beta * U_t+1 + U_t -  ( C_t ^ theta *  ( 1 - L_t )  ^  ( 1 - theta )  )  ^  ( 1 - tau )  /  ( 1 - tau ) '